In [1]:
import json
import os
import tqdm
import numpy as np
import math
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization, Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

from scripts.layers import Spectrogram, MelSpectrogram, LogMelSpectrogram
from scripts.metrics import Precision, Recall, F1
from scripts.data_generator import DataGenerator

### Get training data stats and parameters

In [ ]:
train_path = 'data/train/'
nr_x_files = int(len(os.listdir(train_path)) / 2)
x_filepaths = [train_path + f'X{i}.npy' for i in range(nr_x_files)]

data_stats = {}
means = []
stds = []
mins = []
maxs = []

for filepath in tqdm.tqdm(x_filepaths):
    x = np.load(filepath)

    means.append(x.mean())
    stds.append(x.std())
    mins.append(x.min())
    maxs.append(x.max())

data_stats['x_min'] = min(mins)
data_stats['x_max'] = max(maxs)
data_stats['data_mean'] = np.mean(means)
data_stats['data_std'] = np.mean(stds)

# Data Generators require the data stats
with open('stats/data_stats.json', 'w') as outfile:
    json.dump(data_stats, outfile)

In [2]:
# load params and hparams
with open('stats/params.json') as f:
    params = json.load(f)

with open('stats/hparams.json') as f:
    hparams = json.load(f)

# get additional data stats
with open('stats/data_stats.json') as json_file:
    data_stats = json.load(json_file)

params.update(data_stats)

### Data Generators

In [3]:
# get file paths for each set
train_path = 'data/train/'
val_path = 'data/val/'
test_path = 'data/test/'

nr_train_files = int(len(os.listdir(train_path)) / 2)
nr_val_files = int(len(os.listdir(val_path)) / 2)
nr_test_files = int(len(os.listdir(test_path)) / 2)

filenames_X_train = [train_path + f'X{i}.npy' for i in range(nr_train_files)]
filenames_X_val = [val_path + f'X{i}.npy' for i in range(nr_val_files)]
filenames_X_test = [test_path + f'X{i}.npy' for i in range(nr_test_files)]

filenames_y_train = [train_path + f'y{i}.npy' for i in range(nr_train_files)]
filenames_y_val = [val_path + f'y{i}.npy' for i in range(nr_val_files)]
filenames_y_test = [test_path + f'y{i}.npy' for i in range(nr_test_files)]

In [4]:
# data generators
train_generator = DataGenerator(filenames_X_train, filenames_y_train)
val_generator = DataGenerator(filenames_X_val, filenames_y_val)
test_generator = DataGenerator(filenames_X_test, filenames_y_test)

### Callbacks

In [5]:
# create callback directory
callback_dir = 'callbacks/'
if not os.path.isdir(callback_dir):
        os.mkdir(callback_dir)

# create check pointer
nr_batches = 336
model_path = callback_dir + '/model-best.h5'
checkpointer = ModelCheckpoint(
    model_path,
    save_best_only=True,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose=True,
    save_freq=nr_batches * 10  # number of batches to save progress. - every 10 epochs = nr_batches * nr_epochs
)

# create early stopper
earlystopper = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=20,
    verbose=1,
    restore_best_weights=True
)


def step_decay(epoch):
    initial_lrate = hparams['lr']
    drop = 0.5
    epochs_drop = 10.0

    if epoch < 10:
        return initial_lrate

    elif epoch % 10 == 0:
        lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))

        if lrate < 1e-5:
            lrate = 1e-5

        print(f'\nChanging learning rate to {lrate}\n')

        return lrate

    else:
        lrate = initial_lrate * math.pow(drop,
                                         math.floor((1+epoch - (epoch % 10))/epochs_drop))
        return lrate

lr_scheduler = LearningRateScheduler(step_decay)

callbacks = [checkpointer, earlystopper, lr_scheduler]

### Model

In [8]:
# load saved model
model = tf.keras.models.load_model('saved_model/init_model',
                                   custom_objects={'Precision': Precision,
                                                   'Recall': Recall,
                                                   'F1': F1})

print(model.summary())


Model: "Trigger-word-Marvin-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocessing_layer (Sequen  (None, 86, 64)           0         
 tial)                                                           
                                                                 
 conv_layer (Sequential)     (None, 43, 32)            2080      
                                                                 
 rnn_layer (Sequential)      (None, 128)               360960    
                                                                 
 dense_layer (Sequential)    (None, 1)                 129       
                                                                 
Total params: 363,169
Trainable params: 362,401
Non-trainable params: 768
_________________________________________________________________
None


### Model training

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=hparams['epochs'],
    callbacks=callbacks
)

In [ ]:
# evaluation
model.evaluate(test_generator)

### Visualisation

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 4))

ax[0].set_title('Model loss')
ax[0].plot(history.history['loss'], label='train')
ax[0].plot(history.history['val_loss'], label='val')
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('loss')
ax[0].legend()

ax[1].set_title('Precision')
ax[1].plot(history.history['Precision'], label='train')
ax[1].plot(history.history['val_Precision'], label='val')
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('precision')
ax[1].legend()

ax[2].set_title('Recall')
ax[2].plot(history.history['Recall'], label='train')
ax[2].plot(history.history['val_Recall'], label='val')
ax[2].set_xlabel('epoch')
ax[2].set_ylabel('recall')
ax[2].legend()

ax[3].set_title('F1')
ax[3].plot(history.history['F1'], label='train')
ax[3].plot(history.history['val_F1'], label='val')
ax[3].set_xlabel('epoch')
ax[3].set_ylabel('f1')
ax[3].legend()

plt.show()

In [ ]:
# save model
model_dir = 'models/'

# serialize model to json
model_json = model.to_json()

with open(model_dir + 'marvin-model-struct.json', 'w') as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights(model_dir + "marvin-model-weights.h5")

# SavedModel format
model.save('saved_model/trained_model')


